In [49]:
import zero2one

from collections import Counter
import pandas as pd
from tqdm.notebook import tqdm

from nltk import collocations
from konlpy.tag import Twitter,Okt,Kkma
import ast
okt = Okt()
kkma = Kkma()

In [50]:
def tokenization_kkma(df_,col_):
    senten_2 = []
    tq = tqdm(df_[col_])
    for sen_ in tq:
        sen_3 = list(map(lambda x: kkma.pos(x),sen_))
        senten_2.append(list(map(lambda x: [x_2[0] for x_2 in x],sen_3)))

    df_["sentence_kkma"] = senten_2
    return df_

# 1. Data preprocessing

### - tokenization process: Okt --> kkma

## 1.2 review data

In [51]:
#td1_1 = []
#path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping\coupang"

td1_1 = ["Noun","Verb","Adverb","Adjective","Unknown","ETC","VerbPrefix"]
path_ = r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\hair\shopping_2\test"

test_ = zero2one.Data_Load("contents",path_,"xlsx").parsing_date("date")
test_pos = zero2one.Pos_Process(test_,"contents").pos_tagging(td1_1,"okt")

Split Sentences:   0%|          | 0/171 [00:00<?, ?it/s]

Pos Tagging:   0%|          | 0/171 [00:00<?, ?it/s]

In [52]:
df_ = zero2one.Model1(test_pos,"sen_word2","sen_tag2").df
df_.head()

Preprocessing:   0%|          | 0/171 [00:00<?, ?it/s]

,date,id,Product Name,score,liked,sentence,sentag,review_num,sen_num,channel
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,[좋다],"[(좋다, Adjective)]",1,1,Auction
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[늘, 쓰다, 헤어드라이어, 고장, 나서다, 삿어]","[(늘, Noun), (쓰다, Verb), (헤어드라이어, Noun), (고장, N...",1,2,Auction
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[가격, 좋다, 한일, 좋다]","[(가격, Noun), (좋다, Adjective), (한일, Noun), (좋다,...",1,3,Auction
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[자다, 용하다, 가볍다]","[(자다, Verb), (용하다, Adjective), (가볍다, Adjective)]",1,4,Auction
4,2021.02.26,jsy3***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,[만족스럽다],"[(만족스럽다, Adjective)]",2,1,Auction


In [53]:
df_ = tokenization_kkma(df_,"sentence")
df_.head()

  0%|          | 0/335 [00:00<?, ?it/s]

,date,id,Product Name,score,liked,sentence,sentag,review_num,sen_num,channel,sentence_kkma
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,[좋다],"[(좋다, Adjective)]",1,1,Auction,"[[좋, 다]]"
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[늘, 쓰다, 헤어드라이어, 고장, 나서다, 삿어]","[(늘, Noun), (쓰다, Verb), (헤어드라이어, Noun), (고장, N...",1,2,Auction,"[[늘], [쓰, 다], [헤어드라이어], [고장], [나, 서, 닿], [삿, 얼]]"
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[가격, 좋다, 한일, 좋다]","[(가격, Noun), (좋다, Adjective), (한일, Noun), (좋다,...",1,3,Auction,"[[가격], [좋, 다], [한일], [좋, 다]]"
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,"[자다, 용하다, 가볍다]","[(자다, Verb), (용하다, Adjective), (가볍다, Adjective)]",1,4,Auction,"[[자, 아, 달], [용하, 다], [가볍, 다]]"
4,2021.02.26,jsy3***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,100,0,[만족스럽다],"[(만족스럽다, Adjective)]",2,1,Auction,"[[만족, 스럽, 다]]"


In [54]:
def review_split(df_):
    date_li,id_li,product_li,re_n,sen_n,sentence_list,sentence_list_2,sentag_list,channel_list = [],[],[],[],[],[],[],[],[]
    tq = tqdm(range(len(df_)))
    r = 1
    for i in tq:
        sentences_ = df_.loc[i,"sentence"]
        sentences_2 = df_.loc[i,"sentence_kkma"]
        sentags_ = df_.loc[i,"sentag"]
        for i2 in range(len(sentences_)):
            date_li.append(df_["date"][i])
            id_li.append(df_["id"][i])
            product_li.append(df_["Product Name"][i])
            re_n.append(df_["review_num"][i])
            sen_n.append(df_["sen_num"][i])
            channel_list.append(df_["channel"][i])
            sentence_list.append(sentences_[i2])
            sentence_list_2.append(sentences_2[i2])
            sentag_list.append(sentags_[i2])
            
    da = [date_li,id_li,product_li,re_n,sen_n,sentence_list,sentence_list_2,sentag_list,channel_list]
    df_2 = pd.DataFrame(da).T.rename(columns={0:"date",1:"id",2:"Product Name",
                                              3:"review_num",4:"sen_num",5:"sentence_okt",6:"sentence_kkma",7:"sentag",8:"channel"})
    return df_2

In [55]:
df_2 = review_split(df_)
df_2.head()

  0%|          | 0/335 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction


In [56]:
def review_tag_query(df_2,td1_1,col_):
    tq = tqdm(range(len(df_2)))
    drop_index = []    
    for i in tq:
        sub_tag = df_2.loc[i,col_][1]
        if sub_tag in td1_1:
            pass
        else:
            drop_index.append(i)
    df_3 = df_2.drop(drop_index,axis=0).reset_index(drop=True)
    ### reason numbering
    sen_num = 1
    reason_num = 1
    reason_num_list = []
    for i2 in range(len(df_3)):
        sen_num_n = df_3["sen_num"][i2]
        if sen_num == sen_num_n:
            reason_num_list.append(reason_num)
            reason_num += 1
        else:
            reason_num = 1
            reason_num_list.append(reason_num)
            sen_num += 1
            reason_num += 1
    df_3["reason_num"] = reason_num_list
    return df_3

In [57]:
target_tag = ["Noun","Verb","Adjective","Adverb","VerbPrefix"]
review_df = review_tag_query(df_2,target_tag,"sentag")
review_df.head()

  0%|          | 0/2015 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4


# Delete stop words

In [58]:
# 제가 몇개만 추려서 파일을 만들었습니다.
stop_df = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\result\0319\stop_df.xlsx")
stop_df["reason"] = stop_df["reason"].map(lambda x : ast.literal_eval(x))
stop_df.head()

,reason
0,[하다]
1,[아주]
2,[이다]
3,[아직도]
4,[있다]


In [59]:
# 지우기
def stop_remove(df_total,stop_df,version_):
    stop_index = []
    
    stop_list = []
    for i in range(len(stop_df)):
        stop_list.extend(stop_df.loc[i,"reason"])
    
    if version_ == "okt":
        for row,col in df_total.iterrows():
            if col["sentence_okt"] != "":
                if set(col["sentence_okt"]).issubset(stop_list):
                    #print(col["reason"])
                    stop_index.append(row)
                    
    elif version_ == "kkma":
        stop_list = list(map(lambda x : [x2[0] for x2 in kkma.pos(x)],stop_list))
        
        for row,col in df_total.iterrows():
            if col["sentence_kkma"] != "":
                if col["sentence_kkma"] in stop_list:
                    #print(col["reason"])
                    stop_index.append(row)
        
    df_total = df_total.drop(stop_index,axis=0).reset_index(drop=True)
    return df_total,stop_list

In [60]:
review_df,stop_li = stop_remove(review_df,stop_df,"kkma")


## 1.2 Sentiment_dic 

In [61]:
#dic_ = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\Reference_DB\Sentiment_DB\군산대\SentiWord_Dict_1.xlsx")
dic_ = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\Reference_DB\Sentiment_DB\customizing\SentiWord_Dict_0226.xlsx")

In [62]:
def sentiment_prepro(dic_,traget_tag,remove_list,version_):
    if version_ == "okt":
        dic_["tagging"] = dic_["Senti_Word"].map(lambda x : okt.pos(x,stem=True))
    elif version_ == "kkma":
        dic_["tagging"] = dic_["Senti_Word"].map(lambda x : kkma.pos(x))
    
    if target_tag != []:
        dic_["tagging_only"] = dic_["tagging"].map(lambda x : [ x2[0] for x2 in x if x2[1] in traget_tag ] )
    else :
        dic_["tagging_only"] = dic_["tagging"].map(lambda x : [ x2[0] for x2 in x ] )
    
    dic_["len"] = dic_["tagging_only"].map(lambda x : len(x))
    dic_ = dic_[dic_["len"] != 0]
    dic_["tags"] = dic_["tagging"].map(lambda x: [x2[1] for x2 in x])
    #dic_["tags_2"] = dic_["tags"].map(lambda x: ",".join(x))
    remove_index = dic_.tagging_only.apply(lambda x: set(x).issubset(remove_list))
    # remove_index 에 있는 True를 False로 False를 True로 바꾸는 작업
    dic_ = dic_[remove_index.apply(lambda x: True if x==False else False)]
    return dic_.drop(columns="Unnamed: 0").reset_index(drop=True)

In [63]:
#td1_1 = ["Noun","Verb","Adverb","Adjective","Unknown","ETC","VerbPrefix"]

target_tag = ["Noun","Verb","Adjective","Adverb","VerbPrefix"]

remove_list = ["하다","저","없는단어무언가"]
senti_dic = sentiment_prepro(dic_,target_tag,remove_list,"okt")
senti_dic.head()

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len,tags
0,가능,긍정,1,NaN,"[(가능, Noun)]",[가능],1,[Noun]
1,가능하다,긍정,1,NaN,"[(가능하다, Adjective)]",[가능하다],1,[Adjective]
2,감동,긍정,1,NaN,"[(감동, Noun)]",[감동],1,[Noun]
3,감사,긍정,1,NaN,"[(감사, Noun)]",[감사],1,[Noun]
4,감사하다,긍정,1,NaN,"[(감사하다, Verb)]",[감사하다],1,[Verb]


In [64]:
senti_df = tokenization_kkma(senti_dic,"tagging_only")
senti_df.head()

  0%|          | 0/625 [00:00<?, ?it/s]

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len,tags,sentence_kkma
0,가능,긍정,1,NaN,"[(가능, Noun)]",[가능],1,[Noun],[[가능]]
1,가능하다,긍정,1,NaN,"[(가능하다, Adjective)]",[가능하다],1,[Adjective],"[[가능, 하, 다]]"
2,감동,긍정,1,NaN,"[(감동, Noun)]",[감동],1,[Noun],[[감동]]
3,감사,긍정,1,NaN,"[(감사, Noun)]",[감사],1,[Noun],[[감사]]
4,감사하다,긍정,1,NaN,"[(감사하다, Verb)]",[감사하다],1,[Verb],"[[감사, 하, 다]]"


In [65]:
senti_df["sent_2_len"] = senti_df["sentence_kkma"].map(lambda x: len(x))
senti_df[senti_df["sent_2_len"]==2]

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len,tags,sentence_kkma,sent_2_len
45,맘에든다,긍정,1,NaN,"[(맘, Noun), (에든, Josa), (다, Adverb)]","[맘, 다]",2,"[Noun, Josa, Adverb]","[[맘], [다]]",2
592,좋앙,긍정,1,NaN,"[(좋다, Adjective), (앙, Adverb)]","[좋다, 앙]",2,"[Adjective, Adverb]","[[좋, 다], [앙]]",2
622,못해,부정,-1,NaN,"[(못, VerbPrefix), (하다, Verb)]","[못, 하다]",2,"[VerbPrefix, Verb]","[[못], [하다]]",2
623,못함,부정,-1,NaN,"[(못, VerbPrefix), (하다, Verb)]","[못, 하다]",2,"[VerbPrefix, Verb]","[[못], [하다]]",2
624,안합,부정,-1,NaN,"[(안, VerbPrefix), (하다, Verb)]","[안, 하다]",2,"[VerbPrefix, Verb]","[[안], [하다]]",2


In [66]:
senti_df[senti_df["sent_2_len"]==3]

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len,tags,sentence_kkma,sent_2_len
621,안접힘,부정,-1,NaN,"[(안, VerbPrefix), (접다, Verb), (힘, Noun)]","[안, 접다, 힘]",3,"[VerbPrefix, Verb, Noun]","[[안], [접, 다], [힘]]",3


## 1.3 zero2one_ User

In [67]:
u_dic = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\Reference_DB\Sentiment_DB\usage_dic.xlsx")
u_dic["com"] = u_dic["com"].map(lambda x : ast.literal_eval(x))
u_dic

,key,attr,sim,com
0,사용 대상,자녀,#아기 #딸 #아들 #아이,"[자녀, 아기, 딸, 아들, 아이]"
1,사용 대상,강아지,#반려동물 #개,"[강아지, 개, 반려동물, 반려 동물]"
2,사용 대상,여성,#여자 #엄마,"[여성, 여자, 엄마]"
3,사용 대상,남성,#남자 #아빠,"[남성, 남자, 아빠]"
4,사용 장소,화장대,#책상,"[화장대, 책상]"
5,사용 장소,화장실,#욕실,"[화장실, 욕실]"
6,사용 장소,수영장,#목욕탕,"[수영장, 목욕탕]"
7,사용 장소,여행,#여행용,"[여행, 여행용]"
8,사용 장소,차량,#이동중 #차 안,"[차량, 이동중, 차 안, 이동 중, 운전]"


In [68]:
user_df = tokenization_kkma(u_dic,"com")
user_df

  0%|          | 0/9 [00:00<?, ?it/s]

,key,attr,sim,com,sentence_kkma
0,사용 대상,자녀,#아기 #딸 #아들 #아이,"[자녀, 아기, 딸, 아들, 아이]","[[자녀], [아기], [딸], [아들], [아이]]"
1,사용 대상,강아지,#반려동물 #개,"[강아지, 개, 반려동물, 반려 동물]","[[강아지], [개], [반려, 동물], [반려, 동물]]"
2,사용 대상,여성,#여자 #엄마,"[여성, 여자, 엄마]","[[여성], [여자], [엄마]]"
3,사용 대상,남성,#남자 #아빠,"[남성, 남자, 아빠]","[[남성], [남자], [아빠]]"
4,사용 장소,화장대,#책상,"[화장대, 책상]","[[화장대], [책상]]"
5,사용 장소,화장실,#욕실,"[화장실, 욕실]","[[화장실], [욕실]]"
6,사용 장소,수영장,#목욕탕,"[수영장, 목욕탕]","[[수영장], [목욕탕]]"
7,사용 장소,여행,#여행용,"[여행, 여행용]","[[여행], [여행용]]"
8,사용 장소,차량,#이동중 #차 안,"[차량, 이동중, 차 안, 이동 중, 운전]","[[차량], [이동, 중], [차, 안], [이동, 중], [운전]]"


## 1.4 zero2one Seed

In [69]:
zero_df = pd.read_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\dic_test\드라이기_20201207.xlsx",sheet_name="data_")
zero_df.head()

,Unnamed: 0,Product Attribute 측면 속성,유사 키워드(attr_sim),토큰화 방식으로 각 속성을 나타내는 키워드 조합,count,긍정,부정
0,바람,바람 세기,#바람 강도 #바람 파워 #드라이기 파워 #건조 속도,"['바람 세기', '바람 강하다', '바람 세다', '바람 쎄다', '바람 약하다'...",20916,"바람도 쎄고, 드라이기 바람세기 만족해요, 바람도 세서 좋아요, 바람세기도 좋아서...","바람이 약해요, 바람도 세지도 않아요, 생각만큼 바람이 쎄지는 않습니다, 바람이 흩..."
1,바람,풍량 조절,#풍속 조절 #바람강도 조절 #강도 조절 #바람 세기 조절,"['풍량 조절', '풍속 조절', '바람 강도 조절', '강도 조절', '바람 세기...",1002,"바람세기 조절도 가능하고요, 강약 조절도 잘 되는거 같습니다, 3단계 버튼 조절이 ...","단계조절이 안되는 것같아서, 풍량조절부분이 뻑뻑하네요, 바람 세기 조절이 강약 뿐인..."
2,바람,바람 회전,#토네이도 #회전 바람,"['바람 회전', '토네이도']",28,바람 나오는 부위도 360회전이 되니 사용할때 내가 편한대로 할수 잇어서 더 좋아여...,괴물 토네이도 이런 표현은좀 오바네요
3,온도,냉풍,#차가운 바람 #찬바람,"['냉풍', '차가운 바람', '찬바람', '바람 차갑다']",2476,"가장 큰 장점은 냉풍이에요, 냉풍이라 머릿결도 상하지 않을거 같고 너무 좋네요, ...","냉풍은 더운바람 나옵니다, 냉풍이라고하지만 약간 따뜻한 바람이나오네요, 냉풍이 별로..."
4,온도,온풍,#따뜻한 바람 #부드러운 바람,"['온풍', '따뜻하다 바람', '부드럽다 바람']",2529,"파워 온풍, 따뜻한 바람이 잘 나와서 좋네요, 따뜻한 바람 강력, 따뜻한 바람 적...","온풍으로는 두피나 헤어 손상이 걱정됩니다, 너무 뜨거운듯, 따뜻한 바람을 별로 좋아..."


In [70]:
def remove_list(ta_list,re_list):
    for re in re_list:
        try:
            ta_list.remove(re)
        except:
            pass
    return ta_list

In [71]:
def Zero_dic(zero_df):
    zero_dic = {}
    for i in range(len(zero_df)):
        keys_ = zero_df.iloc[i,0].replace("/"," ")
        keys_li = keys_.split(" ")
        keys_li = remove_list(keys_li,["","및"])


        attrs_ = zero_df.iloc[i,1]
        attrs_li = attrs_.split(" ")
        attrs_li = remove_list(attrs_li,["","및","사용"])

        all_li = keys_li + attrs_li
        try:
            zero_dic[keys_] = list(set(zero_dic[keys_] + all_li))
        except:
            zero_dic[keys_] = list(set(all_li))
    
    all_ma = []
    for key,value in zero_dic.items():
        all_ma.extend(value)    
    
    
    key_li,value_li = [],[]
    
    for key,value in zero_dic.items():
        key_li.append(key)
        value_li.append(value)
    
    zero_df = pd.DataFrame([key_li,value_li]).T.rename(columns={0:"key",1:"aspect"})
    
    return zero_dic,all_ma,zero_df

In [72]:
aspect_dic,aspect_df = Zero_dic(zero_df)[0],Zero_dic(zero_df)[2]
aspect_df

,key,aspect
0,바람,"[세기, 조절, 풍량, 바람, 회전]"
1,온도,"[조절, 냉풍, 온도, 온풍, 열풍, 단계]"
2,조작감,"[조절, 버튼식, COOL버튼, 조작감, 온도, 슬라이드식]"
3,디지털 센서,"[조절, 모발, 디지털, 보호, 모발속, 터치, 온도, 기술, 센싱, 모드, 센서,..."
4,발생 성분,"[성분, 플리즈마, 음이온, 발생, 케어, 이온, 원적외선]"
5,안전 기능,"[기기, 자체, 차단, 미열, 유해, 전자파, 시스템, 물질, 장치, 제어, 기능,..."
6,위생,"[케이스, 탈착필터, 흡인구, 위생, 분리]"
7,스타일링 부속품,"[스무딩, 디퓨저, 집중노즐, 노즐, 부착형, 부속품, 스타일링, 콘센트레이터, 자석]"
8,손잡이 형태,"[접이식, 형태, 손잡이, 고정식]"
9,코드,"[길이, 코드, 무선, 코드릴, 연질]"


In [73]:
aspect_df = tokenization_kkma(aspect_df,"aspect")
aspect_df.head()

  0%|          | 0/21 [00:00<?, ?it/s]

,key,aspect,sentence_kkma
0,바람,"[세기, 조절, 풍량, 바람, 회전]","[[세기], [조절], [풍, 량], [바람], [회전]]"
1,온도,"[조절, 냉풍, 온도, 온풍, 열풍, 단계]","[[조절], [냉풍], [온도], [온풍], [열풍], [단계]]"
2,조작감,"[조절, 버튼식, COOL버튼, 조작감, 온도, 슬라이드식]","[[조절], [버튼, 식], [COOL, 버튼], [조작, 감], [온도], [슬라..."
3,디지털 센서,"[조절, 모발, 디지털, 보호, 모발속, 터치, 온도, 기술, 센싱, 모드, 센서,...","[[조절], [모발], [디지털], [보호], [모발, 속], [터치], [온도],..."
4,발생 성분,"[성분, 플리즈마, 음이온, 발생, 케어, 이온, 원적외선]","[[성분], [플, 리, 즈, 마], [음이온], [발생], [케어], [이온], ..."


# 2) Matching Process

In [74]:
## 다음 sentiment 단어가 나오지 전까지 채우는 작업 
def filling_(df_ma,target_col):
    df_reverse = df_ma.iloc[::-1].copy().reset_index(drop=True)
    tq = tqdm(range(len(df_reverse)))
    s_li = []
    buffer_ = (0,0,None)
    for i in tq:
        rn_ = df_reverse.loc[i,"review_num"]
        sn_ = df_reverse.loc[i,"sen_num"]
        s_ = df_reverse.loc[i,target_col]
        if s_ != []:
            buffer_ = (rn_,sn_,s_)
            s_li.append(buffer_[2])
        else:
            if rn_ == buffer_[0]:
                if sn_ == buffer_[1]:
                    s_li.append(buffer_[2])
                else:
                    s_li.append([])
            else:
                buffer_ = (0,0,None)
                s_li.append([])
    df_reverse[target_col] = s_li
    df_3 = df_reverse.iloc[::-1].copy().reset_index(drop=True)
    return df_3

In [75]:
def filling_neu_(df_fill,target_col,null_data):
    neu_list = []
    for i in range(len(df_fill)):
        s_ = df_fill.loc[i,target_col]
        if s_ == []:
            neu_list.append(null_data)
        else:
            neu_list.append(s_)
    df_fill[target_col] = neu_list
    return df_fill

## 2.1 match betweens "review_df" & "senti_df" 

- for filling columns "s"

In [76]:
review_df.head()

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4


In [77]:
senti_df.head()

,Senti_Word,Orientation,Intensity,Type,tagging,tagging_only,len,tags,sentence_kkma,sent_2_len
0,가능,긍정,1,NaN,"[(가능, Noun)]",[가능],1,[Noun],[[가능]],1
1,가능하다,긍정,1,NaN,"[(가능하다, Adjective)]",[가능하다],1,[Adjective],"[[가능, 하, 다]]",1
2,감동,긍정,1,NaN,"[(감동, Noun)]",[감동],1,[Noun],[[감동]],1
3,감사,긍정,1,NaN,"[(감사, Noun)]",[감사],1,[Noun],[[감사]],1
4,감사하다,긍정,1,NaN,"[(감사하다, Verb)]",[감사하다],1,[Verb],"[[감사, 하, 다]]",1


# * review의 sentence_kkma는 무조건 하나,

--> senti가 항상 review를 포함한다

In [78]:
def query_out(senti_df,reason_all):
    reason = reason_all["sentence_kkma"]
    try:
        query = senti_df[senti_df.sentence_kkma.apply(lambda x: True if reason in x else False)]
        if len(query) !=0:
            try:
                n_nega = query["Orientation"].value_counts()["부정"]
            except:
                n_nega = 0
            try:
                n_posi = query["Orientation"].value_counts()["긍정"]
            except:
                n_posi = 0
            if n_nega == n_posi:
                senti_q = [reason_all["sentence_okt"],"중립",0]
            else:
                senti_q = [reason_all["sentence_okt"],query["Orientation"].mode().values[0],query["Intensity"].mode().values[0],len(query)]
        else:
            senti_q = []
    except:
        senti_q = []
    return senti_q

In [79]:
def matching_s(review_df,senti_df):
    s_li = []
    tq = tqdm(range(len(review_df)))
    for t in tq:
        reason_all = review_df.loc[t]
        s_li.append(query_out(senti_df,reason_all))
    match_1 = review_df.copy()
    match_1["s"] = s_li
    return match_1

In [80]:
match_1 = matching_s(review_df,senti_df)
match_1.loc[0:20]

  0%|          | 0/1792 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]"
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,[]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,[]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,[]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]"
5,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,나서다,"[나, 서, 닿]","(나서다, Verb)",Auction,5,[]
6,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,삿어,"[삿, 얼]","(삿어, Noun)",Auction,6,[]
7,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,가격,[가격],"(가격, Noun)",Auction,1,[]
8,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,좋다,"[좋, 다]","(좋다, Adjective)",Auction,2,"[좋다, 긍정, 1, 3]"
9,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,한일,[한일],"(한일, Noun)",Auction,3,[]


In [81]:
match_1_f = filling_(match_1,"s")
match_1_n = filling_neu_(match_1_f,"s",["Null","중립",0])
match_1_n.head()

  0%|          | 0/1792 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]"
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]"
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]"
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]"
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]"


## 2.2 matching between "review_df" & "user_df"

In [82]:
match_1_n.head()

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]"
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]"
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]"
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]"
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]"


In [83]:
user_df

,key,attr,sim,com,sentence_kkma
0,사용 대상,자녀,#아기 #딸 #아들 #아이,"[자녀, 아기, 딸, 아들, 아이]","[[자녀], [아기], [딸], [아들], [아이]]"
1,사용 대상,강아지,#반려동물 #개,"[강아지, 개, 반려동물, 반려 동물]","[[강아지], [개], [반려, 동물], [반려, 동물]]"
2,사용 대상,여성,#여자 #엄마,"[여성, 여자, 엄마]","[[여성], [여자], [엄마]]"
3,사용 대상,남성,#남자 #아빠,"[남성, 남자, 아빠]","[[남성], [남자], [아빠]]"
4,사용 장소,화장대,#책상,"[화장대, 책상]","[[화장대], [책상]]"
5,사용 장소,화장실,#욕실,"[화장실, 욕실]","[[화장실], [욕실]]"
6,사용 장소,수영장,#목욕탕,"[수영장, 목욕탕]","[[수영장], [목욕탕]]"
7,사용 장소,여행,#여행용,"[여행, 여행용]","[[여행], [여행용]]"
8,사용 장소,차량,#이동중 #차 안,"[차량, 이동중, 차 안, 이동 중, 운전]","[[차량], [이동, 중], [차, 안], [이동, 중], [운전]]"


In [84]:
reason = ["자녀"]
user_df[user_df.sentence_kkma.apply(lambda x: True if reason in x else False)]["attr"].mode().values.tolist()


['자녀 ']

In [85]:
def matching_u(review_df,user_df):
    s_li = []
    tq = tqdm(range(len(review_df)))
    for t in tq:
        reason_all = review_df.loc[t]
        reason = reason_all["sentence_kkma"]
        query_ = user_df[user_df.sentence_kkma.apply(lambda x: True if reason in x else False)]["attr"].mode().values.tolist()
        s_li.append(query_)
    match_1 = review_df.copy()
    match_1["u"] = s_li
    return match_1

In [86]:
match_2_1 = matching_u(match_1_n,user_df)

  0%|          | 0/1792 [00:00<?, ?it/s]

In [87]:
match_2_2 = filling_(match_2_1,"u")
match_2 = filling_neu_(match_2_2,"u",[])
match_2.head()

  0%|          | 0/1792 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s,u
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]",[]
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]",[]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]",[]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]",[]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]",[]


## 2.3 matching between "review_df" & "aspect_df"

In [88]:
match_2.head()

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s,u
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]",[]
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]",[]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]",[]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]",[]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]",[]


In [89]:
aspect_df.head()

,key,aspect,sentence_kkma
0,바람,"[세기, 조절, 풍량, 바람, 회전]","[[세기], [조절], [풍, 량], [바람], [회전]]"
1,온도,"[조절, 냉풍, 온도, 온풍, 열풍, 단계]","[[조절], [냉풍], [온도], [온풍], [열풍], [단계]]"
2,조작감,"[조절, 버튼식, COOL버튼, 조작감, 온도, 슬라이드식]","[[조절], [버튼, 식], [COOL, 버튼], [조작, 감], [온도], [슬라..."
3,디지털 센서,"[조절, 모발, 디지털, 보호, 모발속, 터치, 온도, 기술, 센싱, 모드, 센서,...","[[조절], [모발], [디지털], [보호], [모발, 속], [터치], [온도],..."
4,발생 성분,"[성분, 플리즈마, 음이온, 발생, 케어, 이온, 원적외선]","[[성분], [플, 리, 즈, 마], [음이온], [발생], [케어], [이온], ..."


In [90]:
def matching_a(review_df,aspect_df):
    s_li = []
    tq = tqdm(range(len(review_df)))
    for t in tq:
        reason_all = review_df.loc[t]
        reason = reason_all["sentence_kkma"]
        query_ = aspect_df[aspect_df.sentence_kkma.apply(lambda x: True if reason in x else False)]["key"].mode().values.tolist()
        s_li.append(query_)
    match_1 = review_df.copy()
    match_1["a"] = s_li
    return match_1

In [91]:
match_3_1 = matching_a(match_2,aspect_df)

  0%|          | 0/1792 [00:00<?, ?it/s]

In [92]:
match_3_2 = filling_(match_3_1,"a")
match_3 = filling_neu_(match_3_2,"a",["드라이기"])
match_3.head()

  0%|          | 0/1792 [00:00<?, ?it/s]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s,u,a
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,좋다,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]",[],[드라이기]
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]",[],[드라이기]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]",[],[드라이기]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]",[],[드라이기]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,고장,[고장],"(고장, Noun)",Auction,4,"[고장, 부정, -1, 1]",[],[드라이기]


# "S", "U" 에 걸렸으면 reason에서 제거

In [93]:
# s 지우기

def delete_reason_s(df_1):
    drop_index = []
    buffer_ = ""
    for row,col in df_1.iterrows():
        
        try:
            if col["sentence_okt"] == col["s"][0]:
                if buffer_ == col["s"][0]:
                    drop_index.append(row)
                else:
                    df_1.loc[row,"sentence_okt"] = ""
                
                #print(col["reason"][0],row)
        except:
            pass
        buffer_ = col["s"][0]
    return df_1.drop(drop_index,axis=0).reset_index(drop=True)

In [94]:
df_drop_s = delete_reason_s(match_3)
df_drop_s.loc[0:8]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s,u,a
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]",[],[드라이기]
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]",[],[드라이기]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]",[],[드라이기]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]",[],[드라이기]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,나서다,"[나, 서, 닿]","(나서다, Verb)",Auction,5,"[Null, 중립, 0]",[],[드라이기]
5,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,삿어,"[삿, 얼]","(삿어, Noun)",Auction,6,"[Null, 중립, 0]",[],[드라이기]
6,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,가격,[가격],"(가격, Noun)",Auction,1,"[좋다, 긍정, 1, 3]",[],[가격 및 혜택]
7,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,한일,[한일],"(한일, Noun)",Auction,3,"[좋다, 긍정, 1, 3]",[],[드라이기]
8,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,4,,"[자, 아, 달]","(자다, Verb)",Auction,1,"[자다, 긍정, 1, 2]",[],[드라이기]


In [95]:
# u 지우기

def delete_reason_u(df_1,user_df):
    drop_index = []
    for row,col in df_1.iterrows():
        try:
            ma_u = user_df[user_df.sentence_kkma.apply(lambda x: True if reason in x else False)]["attr"].mode().values.tolist()
            if ma_u != []:
                if ma_u == col["u"]:
                    #print(ma_u)
                    drop_index.append(row)
                #print(col["reason"][0],row)
        except:
            pass
    return df_1.drop(drop_index,axis=0).reset_index(drop=True)

In [96]:
df_drop = delete_reason_u(df_drop_s,user_df)
df_drop.loc[0:40]

,date,id,Product Name,review_num,sen_num,sentence_okt,sentence_kkma,sentag,channel,reason_num,s,u,a
0,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,1,,"[좋, 다]","(좋다, Adjective)",Auction,1,"[좋다, 긍정, 1, 3]",[],[드라이기]
1,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,늘,[늘],"(늘, Noun)",Auction,1,"[고장, 부정, -1, 1]",[],[드라이기]
2,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,쓰다,"[쓰, 다]","(쓰다, Verb)",Auction,2,"[고장, 부정, -1, 1]",[],[드라이기]
3,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,헤어드라이어,[헤어드라이어],"(헤어드라이어, Noun)",Auction,3,"[고장, 부정, -1, 1]",[],[드라이기]
4,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,나서다,"[나, 서, 닿]","(나서다, Verb)",Auction,5,"[Null, 중립, 0]",[],[드라이기]
5,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,2,삿어,"[삿, 얼]","(삿어, Noun)",Auction,6,"[Null, 중립, 0]",[],[드라이기]
6,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,가격,[가격],"(가격, Noun)",Auction,1,"[좋다, 긍정, 1, 3]",[],[가격 및 혜택]
7,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,3,한일,[한일],"(한일, Noun)",Auction,3,"[좋다, 긍정, 1, 3]",[],[드라이기]
8,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,4,,"[자, 아, 달]","(자다, Verb)",Auction,1,"[자다, 긍정, 1, 2]",[],[드라이기]
9,2021.03.14,dbza***,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,1,4,용하다,"[용하, 다]","(용하다, Adjective)",Auction,2,"[가볍다, 긍정, 1, 1]",[],[드라이기]


# 마지막 전처리

### Opinion_id 처리하기

In [97]:
def all_result(df_total,end_product):
    reason_n = 1
    buffer_ = (1,1,reason_n)
    id_li = []

    for row,col in df_total.iterrows():
        if col["review_num"] == buffer_[0] and col["sen_num"] == buffer_[1]:
            rn_ = str(col["review_num"])
            sn_ = str(col["sen_num"])
            id_ = [end_product,rn_,sn_,str(reason_n)]
            id_li.append("-".join(id_))
        else:
            reason_n = 1
            rn_ = str(col["review_num"])
            if col["review_num"] == buffer_[0] and col["sen_num"] != buffer_[1]+1:
                sn_ = str(buffer_[1]+1)
            else:
                sn_ = str(col["sen_num"])
            id_ = [end_product,rn_,sn_,str(reason_n)]
            id_li.append("-".join(id_))
        buffer_ = (col["review_num"],col["sen_num"],reason_n)
        reason_n += 1
    df_total["Opinion_ID"] = id_li
    new_h_list = []
    for row,col in df_total.iterrows():
        new_h = [col["id"],col["channel"]]
        new_h_list.append(new_h)
    df_total["h"] = new_h_list
    df_total = df_total.rename(columns={"Product Name":"e","sentence_okt":"r","date":"t"})
    df_total = df_total.loc[:,["Opinion_ID","e","a","s","r","u","h","t"]]
    return df_total

In [99]:
end_product = "드라이기"

df_result = all_result(df_drop,end_product)
df_result.loc[0:20]

,Opinion_ID,e,a,s,r,u,h,t
0,드라이기-1-1-1,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[좋다, 긍정, 1, 3]",,[],"[dbza***, Auction]",2021.03.14
1,드라이기-1-2-1,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[고장, 부정, -1, 1]",늘,[],"[dbza***, Auction]",2021.03.14
2,드라이기-1-2-2,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[고장, 부정, -1, 1]",쓰다,[],"[dbza***, Auction]",2021.03.14
3,드라이기-1-2-3,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[고장, 부정, -1, 1]",헤어드라이어,[],"[dbza***, Auction]",2021.03.14
4,드라이기-1-2-4,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[Null, 중립, 0]",나서다,[],"[dbza***, Auction]",2021.03.14
5,드라이기-1-2-5,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[Null, 중립, 0]",삿어,[],"[dbza***, Auction]",2021.03.14
6,드라이기-1-3-1,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[가격 및 혜택],"[좋다, 긍정, 1, 3]",가격,[],"[dbza***, Auction]",2021.03.14
7,드라이기-1-3-2,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[좋다, 긍정, 1, 3]",한일,[],"[dbza***, Auction]",2021.03.14
8,드라이기-1-4-1,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[자다, 긍정, 1, 2]",,[],"[dbza***, Auction]",2021.03.14
9,드라이기-1-4-2,한일_PH-2060_드라이기_헤어드라이기_드라이어_국산.xlsx,[드라이기],"[가볍다, 긍정, 1, 1]",용하다,[],"[dbza***, Auction]",2021.03.14


In [192]:
df_result.to_excel(r"C:\Users\Samsung\jupyter\Project_LG_textmining\data\result\0319\result3.xlsx")